Import the necessary machine learning and utility packages

In [1]:
using Flux,Statistics,Plots,MLDataUtils
using Flux.Data: DataLoader
include("utilityfunc.jl");

Set the upper and lower bounds on the features and define the scaling and unscaling functions

In [2]:
#set lower and upper bounds on Lpt, Kt, Rs,(Lpt is first element, Kt is second element, Rs is the third element)
lb=Float32[5e-7,5e-7,5]
ub=Float32[5e-6,5e-6,30]
conc_ub=Float32(.6)
iscaler(x)=(x-lb)./(ub-lb)
inv_iscaler(x)=x.*(ub.-lb).+lb
oscaler(x)= x ./ conc_ub
inv_oscaler(x)=x.*conc_ub

inv_oscaler (generic function with 1 method)

In [3]:
function iscalerbatch(x)
    nfeat,sl = size(x)
    out=Array{Float32,2}(undef,nfeat,sl)
    for i=1:sl
        out[:,i]=iscaler(x[:,i])
    end
    return out
end

iscalerbatch (generic function with 1 method)

In [4]:
function inv_iscalerbatch(x)
    nfeat,sl = size(x)
    out=Array{Float32,2}(undef,nfeat,sl)
    for i=1:sl
        out[:,i]=inv_iscaler(x[:,i])
    end
    return out
end

inv_iscalerbatch (generic function with 1 method)

Define model characteristics, load the data, reshape the data into proper form, and scale the data

In [5]:
n_samp=Int64(1e4); # number of samples
nfeat=3; #5 features
ntarg=20; #1 target values

In [6]:
d = readdlm("MLFinalData_v3.csv",',',Float32)
X_p= d[1:5,:];
Y_p= d[6,:];
X=Array{Float32,2}(undef,3,n_samp)
for i=1:n_samp
    X[:,i]=X_p[1:3,20*i]
end
Y=reshape(Y_p,(20,n_samp));

In [7]:
X=iscalerbatch(X);
Y = Float32.(oscaler(Y));

Split the data into training, validation, and test sets using 80/10/10 Split

In [8]:
(xtrg, ytrg), (xtg, ytg),(xtest,ytest) = splitobs((X, Y), at = (0.8,.1) );
datatrg=DataLoader((xtrg, ytrg));

Define the MSE loss function

In [9]:
function closs(x,y,nn)
     Flux.mse(nn(x), y)
end

closs (generic function with 1 method)

Set training hyperparameters - 50 epochs, ADAM with a fixed learning rate of 10^-3

Then set the number of features=5,target=1

Execute grid search to assess 18 candidate models for the full integration MLP model. We vary the number of hidden layers (1-2), the activation function (tanh/ReLU/swish), and the number of nodes per hidden layer (8,16,32)

In [10]:
function gridsearchMLP(data_train,x_val,y_val)
    println("Beginning grid search:")
    opt= ADAM(1e-3)
    n_epochs = 50;
    nfeat =3; ntarg =20;
    nh_8,nh_16,nh_32 = 8,16,32;
    m_list = [Chain(Dense(nfeat,nh_8),Dense(nh_8, nh_8,Flux.tanh),Dense(nh_8,ntarg))
    Chain(Dense(nfeat,nh_8),Dense(nh_8, nh_8,Flux.tanh),Dense(nh_8, nh_8,Flux.tanh),Dense(nh_8,ntarg))
        Chain(Dense(nfeat,nh_8),Dense(nh_8, nh_8,Flux.swish),Dense(nh_8,ntarg))
    Chain(Dense(nfeat,nh_8),Dense(nh_8, nh_8,Flux.swish),Dense(nh_8, nh_8,Flux.swish),Dense(nh_8,ntarg))
        Chain(Dense(nfeat,nh_8),Dense(nh_8, nh_8,Flux.relu),Dense(nh_8,ntarg))
    Chain(Dense(nfeat,nh_8),Dense(nh_8, nh_8,Flux.relu),Dense(nh_8, nh_8,Flux.relu),Dense(nh_8,ntarg))
            Chain(Dense(nfeat,nh_16),Dense(nh_16, nh_16,Flux.tanh),Dense(nh_16,ntarg))
    Chain(Dense(nfeat,nh_16),Dense(nh_16, nh_16,Flux.tanh),Dense(nh_16, nh_16,Flux.tanh),Dense(nh_16,ntarg))
        Chain(Dense(nfeat,nh_16),Dense(nh_16, nh_16,Flux.swish),Dense(nh_16,ntarg))
    Chain(Dense(nfeat,nh_16),Dense(nh_16, nh_16,Flux.swish),Dense(nh_16, nh_16,Flux.swish),Dense(nh_16,ntarg))
        Chain(Dense(nfeat,nh_16),Dense(nh_16, nh_16,Flux.relu),Dense(nh_16,ntarg))
    Chain(Dense(nfeat,nh_16),Dense(nh_16, nh_16,Flux.relu),Dense(nh_16, nh_16,Flux.relu),Dense(nh_16,ntarg))
                    Chain(Dense(nfeat,nh_32),Dense(nh_32, nh_32,Flux.tanh),Dense(nh_32,ntarg))
    Chain(Dense(nfeat,nh_32),Dense(nh_32, nh_32,Flux.tanh),Dense(nh_32, nh_32,Flux.tanh),Dense(nh_32,ntarg))
        Chain(Dense(nfeat,nh_32),Dense(nh_32, nh_32,Flux.swish),Dense(nh_32,ntarg))
    Chain(Dense(nfeat,nh_32),Dense(nh_32, nh_32,Flux.swish),Dense(nh_32, nh_32,Flux.swish),Dense(nh_32,ntarg))
        Chain(Dense(nfeat,nh_32),Dense(nh_32, nh_32,Flux.relu),Dense(nh_32,ntarg))
    Chain(Dense(nfeat,nh_32),Dense(nh_32, nh_32,Flux.relu),Dense(nh_32, nh_32,Flux.relu),Dense(nh_32,ntarg))
    ]
    ps_l = [params(x) for x in m_list]
    record = Array{Any,2}(undef,2,18);
    record[1,:]=["8tanh1" "8tanh2" "8swish1" "8swish2" "8relu1" "8relu2"
    "16tanh1" "16tanh2" "16swish1" "16swish2" "16relu1" "16relu2"
        "32tanh1" "32tanh2" "32swish1" "32swish2" "32relu1" "32relu2"]
    for i=1:18
        r=record[1,i]
        println("beginning model $i $r")
        iloss(x,y)=closs(x,y,m_list[i])
        psi = ps_l[i]
        for i=1:50
            Flux.train!(iloss, psi, data_train, opt)
        end
        @show record[2,i]=iloss(x_val,y_val)
    end
    return record
end

gridsearchMLP (generic function with 1 method)

In [ ]:
gridsearchMLP(datatrg,xtg,ytg) #run function

A record of the original grid search run is saved in the file "record of grid search".xlsx -> it is important to note training is computationally expensive and stochastic and that any further runs may take a long time and not match the original results